<a href="https://colab.research.google.com/github/nattntn/PMU-B-PersonalAI/blob/main/Hackathon_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install

In [1]:
!pip install efficientnet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 1.1 MB/s eta 0:00:00


# Load data form Kaggle

> TensorFlow

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Basic
import os
from os import makedirs
from os import listdir
from shutil import copyfile
from random import seed
from random import random
import numpy as np
import pandas as pd
from PIL import Image, ImageFile

# visuals
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.image import imread
from PIL import Image

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,ConfusionMatrixDisplay

# Tensorflow
import tensorflow as tf
from tensorflow.keras import callbacks
from keras.callbacks import Callback
from keras import layers
from keras import models
from tensorflow.keras import optimizers
from keras.optimizers import Adam
from efficientnet.keras import center_crop_and_resize, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
#ดู version ของ tf
import tensorflow as tf
print(tf.__version__)

2.15.0


**Upload API kaggle.json**

In [5]:
! pip install -q kaggle

In [ ]:
#!mkdir -p /content/drive/MyDrive/Hackathon/.kaggle

In [ ]:
# !cp kaggle.json /content/drive/MyDrive/Hackathon/.kaggle/
# !chmod 600 /content/drive/MyDrive/Hackathon/.kaggle/kaggle.json

In [6]:
!mkdir -p ~/.kaggle
# !mkdir -p /content/drive/MyDrive/Hackathon/.kaggle
!cp kaggle.json ~/.kaggle/
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [7]:
data_dir = '/content/drive/MyDrive/Hackathon'
%cd '/content/drive/MyDrive/Hackathon'
!pwd

/content/drive/MyDrive/Hackathon
/content/drive/MyDrive/Hackathon


In [8]:
!kaggle datasets download -d farjanakabirsamanta/skin-cancer-dataset

100% 2.58G/2.58G [01:07<00:00, 58.6MB/s]
100% 2.58G/2.58G [01:07<00:00, 41.2MB/s]


In [9]:
!unzip skin-cancer-dataset.zip

เอาต์พุตของการสตรีมมีการตัดเหลือเพียง 5000 บรรทัดสุดท้าย
  inflating: Skin Cancer/Skin Cancer/ISIC_0029321.jpg  
  inflating: Skin Cancer/Skin Cancer/ISIC_0029322.jpg  
  inflating: Skin Cancer/Skin Cancer/ISIC_0029323.jpg  
  inflating: Skin Cancer/Skin Cancer/ISIC_0029324.jpg  
  inflating: Skin Cancer/Skin Cancer/ISIC_0029325.jpg  
  inflating: Skin Cancer/Skin Cancer/ISIC_0029326.jpg  
  inflating: Skin Cancer/Skin Cancer/ISIC_0029327.jpg  
  inflating: Skin Cancer/Skin Cancer/ISIC_0029328.jpg  
  inflating: Skin Cancer/Skin Cancer/ISIC_0029329.jpg  
  inflating: Skin Cancer/Skin Cancer/ISIC_0029330.jpg  
  inflating: Skin Cancer/Skin Cancer/ISIC_0029331.jpg  
  inflating: Skin Cancer/Skin Cancer/ISIC_0029332.jpg  
  inflating: Skin Cancer/Skin Cancer/ISIC_0029333.jpg  
  inflating: Skin Cancer/Skin Cancer/ISIC_0029334.jpg  
  inflating: Skin Cancer/Skin Cancer/ISIC_0029335.jpg  
  inflating: Skin Cancer/Skin Cancer/ISIC_0029336.jpg  
  inflating: Skin Cancer/Skin Cancer/ISIC_00293

# Create dataframe

In [3]:
#เชื่อม google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## การทำ Path Filename

In [4]:
skin = r'/content/gdrive/MyDrive/Hackathon/Skin Cancer/Skin Cancer' # ให้อ่านที่อยู่ของโฟรเดอร์ในไดร์ฟ และสร้างตัวแปร ชื่อว่า paro7 มารับไว้

In [5]:
import os # os เอาไว้จัดการไฟล์และ path

#unwanted_folders = ['Detail pic.jpg', 'FILM NUMBER.gsheet'] # กำหนดชื่อโฟลเดอร์ที่ไม่ต้องการให้เก็บไฟล์

files = [] # สร้าง list เพื่อเก็บตำแหน่งของไฟล์
for path, dirnames, filenames in os.walk(skin): # os.walk คือ ทำงานสร้างชื่อไฟล์แต่ละไฟล์ ตามที่เก็บข้อมูล
   #if any(folder in path for folder in unwanted_folders): # ตรวจสอบว่า path เป็นโฟลเดอร์ที่ไม่ต้องการหรือไม่
       # continue # ข้ามไปยังโฟลเดอร์ถัดไป
    files.extend(os.path.join(path, name) for name in filenames) # เพิ่มตำแหน่งของไฟล์ลงใน list ถ้าเป็นไฟล์ในโฟลเดอร์ที่ต้องการ # extend  ใช้เพื่อรวม list # คำสั่งเชื่อมส่วนประกอบของ path ที่ประกอบด้วย directory หรือ file เพื่อสร้าง path หรือ URL ที่รองรับการทำงานบนระบบปฏิบัติการต่างๆ
print(files) # แสดงตำแหน่งของไฟล์ทั้งหมดที่อยู่ในโฟลเดอร์ที่ต้องการเก็บ

['/content/gdrive/MyDrive/Hackathon/Skin Cancer/Skin Cancer/ISIC_0033307.jpg', '/content/gdrive/MyDrive/Hackathon/Skin Cancer/Skin Cancer/ISIC_0033345.jpg', '/content/gdrive/MyDrive/Hackathon/Skin Cancer/Skin Cancer/ISIC_0033310.jpg', '/content/gdrive/MyDrive/Hackathon/Skin Cancer/Skin Cancer/ISIC_0033330.jpg', '/content/gdrive/MyDrive/Hackathon/Skin Cancer/Skin Cancer/ISIC_0033328.jpg', '/content/gdrive/MyDrive/Hackathon/Skin Cancer/Skin Cancer/ISIC_0033336.jpg', '/content/gdrive/MyDrive/Hackathon/Skin Cancer/Skin Cancer/ISIC_0033369.jpg', '/content/gdrive/MyDrive/Hackathon/Skin Cancer/Skin Cancer/ISIC_0033371.jpg', '/content/gdrive/MyDrive/Hackathon/Skin Cancer/Skin Cancer/ISIC_0033352.jpg', '/content/gdrive/MyDrive/Hackathon/Skin Cancer/Skin Cancer/ISIC_0033329.jpg', '/content/gdrive/MyDrive/Hackathon/Skin Cancer/Skin Cancer/ISIC_0033360.jpg', '/content/gdrive/MyDrive/Hackathon/Skin Cancer/Skin Cancer/ISIC_0033363.jpg', '/content/gdrive/MyDrive/Hackathon/Skin Cancer/Skin Cancer/ISIC

In [6]:
files

['/content/gdrive/MyDrive/Hackathon/Skin Cancer/Skin Cancer/ISIC_0033307.jpg',
 '/content/gdrive/MyDrive/Hackathon/Skin Cancer/Skin Cancer/ISIC_0033345.jpg',
 '/content/gdrive/MyDrive/Hackathon/Skin Cancer/Skin Cancer/ISIC_0033310.jpg',
 '/content/gdrive/MyDrive/Hackathon/Skin Cancer/Skin Cancer/ISIC_0033330.jpg',
 '/content/gdrive/MyDrive/Hackathon/Skin Cancer/Skin Cancer/ISIC_0033328.jpg',
 '/content/gdrive/MyDrive/Hackathon/Skin Cancer/Skin Cancer/ISIC_0033336.jpg',
 '/content/gdrive/MyDrive/Hackathon/Skin Cancer/Skin Cancer/ISIC_0033369.jpg',
 '/content/gdrive/MyDrive/Hackathon/Skin Cancer/Skin Cancer/ISIC_0033371.jpg',
 '/content/gdrive/MyDrive/Hackathon/Skin Cancer/Skin Cancer/ISIC_0033352.jpg',
 '/content/gdrive/MyDrive/Hackathon/Skin Cancer/Skin Cancer/ISIC_0033329.jpg',
 '/content/gdrive/MyDrive/Hackathon/Skin Cancer/Skin Cancer/ISIC_0033360.jpg',
 '/content/gdrive/MyDrive/Hackathon/Skin Cancer/Skin Cancer/ISIC_0033363.jpg',
 '/content/gdrive/MyDrive/Hackathon/Skin Cancer/Skin

In [7]:
len(files) #  pano images

10015

### ตาราง pathname

In [8]:
import pandas as pd # import ข้อมูลเข้าเพื่อสร้างตาาราง

In [9]:
pathname = pd.DataFrame({'Path_Name':files}) # สร้างตารางคอลลัมม์ที่อยู่ของภาพโดยตั้งชื่อคอลลัมม์ว่า Path_name
pathname #แสดงตาราง

,Path_Name
0,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
1,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
2,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
3,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
4,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
...,...
10010,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
10011,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
10012,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
10013,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...


In [10]:
list_path = list(pathname['Path_Name']) # เก็บที่อยู่ของภาพในรูปแบบตาราง โดยชื่อว่า list_path ซึ่ง list(ชื่อตาราง['ชื่อคอลัมน์'])
for i in list_path: # วนลูปใน list_path
    res = i.split('/') # split ใช้สำหรับแยก โดยในที่นี้แยกด้วย / แล้วตั้งชื่อว่า res
    print(res) # แสดงผลลัพธ์ของ res ตัวที่ 7 คือตำเเหน่งที่เราต้องการ

เอาต์พุตของการสตรีมมีการตัดเหลือเพียง 5000 บรรทัดสุดท้าย
['', 'content', 'gdrive', 'MyDrive', 'Hackathon', 'Skin Cancer', 'Skin Cancer', 'ISIC_0028387.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Hackathon', 'Skin Cancer', 'Skin Cancer', 'ISIC_0028332.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Hackathon', 'Skin Cancer', 'Skin Cancer', 'ISIC_0028347.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Hackathon', 'Skin Cancer', 'Skin Cancer', 'ISIC_0028350.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Hackathon', 'Skin Cancer', 'Skin Cancer', 'ISIC_0028386.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Hackathon', 'Skin Cancer', 'Skin Cancer', 'ISIC_0028385.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Hackathon', 'Skin Cancer', 'Skin Cancer', 'ISIC_0028316.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Hackathon', 'Skin Cancer', 'Skin Cancer', 'ISIC_0028315.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Hackathon', 'Skin Cancer', 'Skin Cancer', 'ISIC_0028353.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Hacka

In [11]:
list_path = list(pathname['Path_Name']) # เก็บที่อยู่ของภาพในรูปแบบตาราง โดยชื่อว่า list_path ซึ่ง list(ชื่อตาราง['ชื่อคอลัมน์'])
class_ima = list()
for i in list_path: # วนลูปใน list_path
    res = i.split('/') # split ใช้สำหรับแยก โดยในที่นี้แยกด้วย / แล้วตั้งชื่อว่า res
    print(res) # แสดงผลลัพธ์ของ res ตัวที่ 7 คือตำเเหน่งที่เราต้องการ
    class_ima.append(res[-1]) # append  คำสั่งเพิ่มข้อมูลลงใน List

pathname['Filename'] = class_ima

เอาต์พุตของการสตรีมมีการตัดเหลือเพียง 5000 บรรทัดสุดท้าย
['', 'content', 'gdrive', 'MyDrive', 'Hackathon', 'Skin Cancer', 'Skin Cancer', 'ISIC_0028387.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Hackathon', 'Skin Cancer', 'Skin Cancer', 'ISIC_0028332.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Hackathon', 'Skin Cancer', 'Skin Cancer', 'ISIC_0028347.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Hackathon', 'Skin Cancer', 'Skin Cancer', 'ISIC_0028350.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Hackathon', 'Skin Cancer', 'Skin Cancer', 'ISIC_0028386.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Hackathon', 'Skin Cancer', 'Skin Cancer', 'ISIC_0028385.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Hackathon', 'Skin Cancer', 'Skin Cancer', 'ISIC_0028316.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Hackathon', 'Skin Cancer', 'Skin Cancer', 'ISIC_0028315.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Hackathon', 'Skin Cancer', 'Skin Cancer', 'ISIC_0028353.jpg']
['', 'content', 'gdrive', 'MyDrive', 'Hacka

In [12]:
pathname

,Path_Name,Filename
0,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...,ISIC_0033307.jpg
1,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...,ISIC_0033345.jpg
2,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...,ISIC_0033310.jpg
3,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...,ISIC_0033330.jpg
4,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...,ISIC_0033328.jpg
...,...,...
10010,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...,ISIC_0024310.jpg
10011,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...,ISIC_0024333.jpg
10012,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...,ISIC_0024335.jpg
10013,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...,ISIC_0024331.jpg


In [ ]:
list_path = list(pathname['Filename']) # เก็บที่อยู่ของภาพในรูปแบบตาราง โดยชื่อว่า list_path ซึ่ง list(ชื่อตาราง['ชื่อคอลัมน์'])
for i in list_path: # วนลูปใน list_path
    res = i.split('.') # split ใช้สำหรับแยก โดยในที่นี้แยกด้วย / แล้วตั้งชื่อว่า res
    print(res) # แสดงผลลัพธ์ของ res ตัวที่ 7 คือตำเเหน่งที่เราต้องการ

In [17]:
list_path = list(pathname['Filename']) # เก็บที่อยู่ของภาพในรูปแบบตาราง โดยชื่อว่า list_path ซึ่ง list(ชื่อตาราง['ชื่อคอลัมน์'])
class_ima = list()
for i in list_path: # วนลูปใน list_path
    res = i.split('.') # split ใช้สำหรับแยก โดยในที่นี้แยกด้วย / แล้วตั้งชื่อว่า res
    print(res) # แสดงผลลัพธ์ของ res ตัวที่ 7 คือตำเเหน่งที่เราต้องการ
    class_ima.append(res[0]) # append  คำสั่งเพิ่มข้อมูลลงใน List

pathname['image_id'] = class_ima

เอาต์พุตของการสตรีมมีการตัดเหลือเพียง 5000 บรรทัดสุดท้าย
['ISIC_0028387', 'jpg']
['ISIC_0028332', 'jpg']
['ISIC_0028347', 'jpg']
['ISIC_0028350', 'jpg']
['ISIC_0028386', 'jpg']
['ISIC_0028385', 'jpg']
['ISIC_0028316', 'jpg']
['ISIC_0028315', 'jpg']
['ISIC_0028353', 'jpg']
['ISIC_0028381', 'jpg']
['ISIC_0028312', 'jpg']
['ISIC_0028391', 'jpg']
['ISIC_0028379', 'jpg']
['ISIC_0028382', 'jpg']
['ISIC_0028345', 'jpg']
['ISIC_0028324', 'jpg']
['ISIC_0028361', 'jpg']
['ISIC_0028388', 'jpg']
['ISIC_0028310', 'jpg']
['ISIC_0028352', 'jpg']
['ISIC_0028376', 'jpg']
['ISIC_0028395', 'jpg']
['ISIC_0028370', 'jpg']
['ISIC_0028375', 'jpg']
['ISIC_0028330', 'jpg']
['ISIC_0028346', 'jpg']
['ISIC_0028343', 'jpg']
['ISIC_0028308', 'jpg']
['ISIC_0028394', 'jpg']
['ISIC_0028360', 'jpg']
['ISIC_0028311', 'jpg']
['ISIC_0028314', 'jpg']
['ISIC_0028335', 'jpg']
['ISIC_0028341', 'jpg']
['ISIC_0028334', 'jpg']
['ISIC_0028325', 'jpg']
['ISIC_0028333', 'jpg']
['ISIC_0028342', 'jpg']
['ISIC_0028369', 'jpg']
['ISIC_

In [18]:
pathname

,Path_Name,Filename,image_id
0,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...,ISIC_0033307.jpg,ISIC_0033307
1,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...,ISIC_0033345.jpg,ISIC_0033345
2,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...,ISIC_0033310.jpg,ISIC_0033310
3,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...,ISIC_0033330.jpg,ISIC_0033330
4,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...,ISIC_0033328.jpg,ISIC_0033328
...,...,...,...
10010,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...,ISIC_0024310.jpg,ISIC_0024310
10011,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...,ISIC_0024333.jpg,ISIC_0024333
10012,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...,ISIC_0024335.jpg,ISIC_0024335
10013,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...,ISIC_0024331.jpg,ISIC_0024331


In [19]:
meta = pd.read_csv(r'/content/gdrive/MyDrive/Hackathon/HAM10000_metadata.csv')
meta

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear
...,...,...,...,...,...,...,...
10010,HAM_0002867,ISIC_0033084,akiec,histo,40.0,male,abdomen
10011,HAM_0002867,ISIC_0033550,akiec,histo,40.0,male,abdomen
10012,HAM_0002867,ISIC_0033536,akiec,histo,40.0,male,abdomen
10013,HAM_0000239,ISIC_0032854,akiec,histo,80.0,male,face


In [20]:
#คีย์หลักซ้ำได้ คีย์รองซ้ำไม่ได้
pathname_nodup = pathname.drop_duplicates(subset=['image_id']) #drop อันซ้ำออกกก่อน
meta['Path_Name'] = meta['image_id'].map(pathname_nodup.set_index('image_id')['Path_Name'])  #เอา  price_min ของตาราง โปรเจค มาใส่ในตาราง unit

In [21]:
meta

,lesion_id,image_id,dx,dx_type,age,sex,localization,Path_Name
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
...,...,...,...,...,...,...,...,...
10010,HAM_0002867,ISIC_0033084,akiec,histo,40.0,male,abdomen,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
10011,HAM_0002867,ISIC_0033550,akiec,histo,40.0,male,abdomen,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
10012,HAM_0002867,ISIC_0033536,akiec,histo,40.0,male,abdomen,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
10013,HAM_0000239,ISIC_0032854,akiec,histo,80.0,male,face,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...


In [24]:
len(set(meta['image_id']))

10015

In [25]:
set(meta['dx']) # มะเร็ง 7 ชนิด

{'akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc'}

In [26]:
print(len(meta[meta['dx']=='akiec']))
print(len(meta[meta['dx']=='bcc']))
print(len(meta[meta['dx']=='bkl']))
print(len(meta[meta['dx']=='df']))# 115
print(len(meta[meta['dx']=='mel']))
print(len(meta[meta['dx']=='nv']))
print(len(meta[meta['dx']=='vasc']))

327
514
1099
115
1113
6705
142


In [28]:
akiec = meta[meta['dx']=='akiec'].iloc[:115,:]
bcc = meta[meta['dx']=='bcc'].iloc[:115,:]
bkl = meta[meta['dx']=='bkl'].iloc[:115,:]
df_dis = meta[meta['dx']=='df']
mel = meta[meta['dx']=='mel'].iloc[:115,:]
nv = meta[meta['dx']=='nv'].iloc[:115,:]
vasc = meta[meta['dx']=='vasc'].iloc[:115,:]

In [29]:
table = pd.concat([akiec,bcc,bkl,df_dis,mel,nv,vasc])
table

,lesion_id,image_id,dx,dx_type,age,sex,localization,Path_Name
9687,HAM_0002644,ISIC_0029417,akiec,histo,80.0,female,neck,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
9688,HAM_0006002,ISIC_0029915,akiec,histo,50.0,female,face,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
9689,HAM_0000549,ISIC_0029360,akiec,histo,70.0,male,upper extremity,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
9690,HAM_0000549,ISIC_0026152,akiec,histo,70.0,male,upper extremity,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
9691,HAM_0000673,ISIC_0029659,akiec,histo,70.0,female,face,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
...,...,...,...,...,...,...,...,...
2430,HAM_0007532,ISIC_0024662,vasc,consensus,75.0,male,trunk,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
2431,HAM_0005885,ISIC_0031215,vasc,consensus,75.0,male,trunk,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
2432,HAM_0007118,ISIC_0027856,vasc,consensus,45.0,female,trunk,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
2433,HAM_0004653,ISIC_0028431,vasc,consensus,50.0,male,abdomen,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...


In [33]:
# table.to_csv('/content/gdrive/MyDrive/Hackathon/train_test_val.csv')

In [31]:
df_another = meta[~meta['image_id'].isin(table['image_id'])]
df_another

,lesion_id,image_id,dx,dx_type,age,sex,localization,Path_Name
116,HAM_0004673,ISIC_0027455,bkl,histo,50.0,male,back,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
117,HAM_0007179,ISIC_0025906,bkl,histo,0.0,female,back,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
118,HAM_0007179,ISIC_0027151,bkl,histo,0.0,female,back,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
119,HAM_0005077,ISIC_0028646,bkl,histo,75.0,male,upper extremity,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
120,HAM_0000566,ISIC_0026794,bkl,histo,60.0,female,face,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
...,...,...,...,...,...,...,...,...
10010,HAM_0002867,ISIC_0033084,akiec,histo,40.0,male,abdomen,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
10011,HAM_0002867,ISIC_0033550,akiec,histo,40.0,male,abdomen,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
10012,HAM_0002867,ISIC_0033536,akiec,histo,40.0,male,abdomen,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
10013,HAM_0000239,ISIC_0032854,akiec,histo,80.0,male,face,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...


In [32]:
# df_another.to_csv('/content/gdrive/MyDrive/Hackathon/df_another.csv')

# Data split

In [34]:
table

,lesion_id,image_id,dx,dx_type,age,sex,localization,Path_Name
9687,HAM_0002644,ISIC_0029417,akiec,histo,80.0,female,neck,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
9688,HAM_0006002,ISIC_0029915,akiec,histo,50.0,female,face,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
9689,HAM_0000549,ISIC_0029360,akiec,histo,70.0,male,upper extremity,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
9690,HAM_0000549,ISIC_0026152,akiec,histo,70.0,male,upper extremity,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
9691,HAM_0000673,ISIC_0029659,akiec,histo,70.0,female,face,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
...,...,...,...,...,...,...,...,...
2430,HAM_0007532,ISIC_0024662,vasc,consensus,75.0,male,trunk,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
2431,HAM_0005885,ISIC_0031215,vasc,consensus,75.0,male,trunk,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
2432,HAM_0007118,ISIC_0027856,vasc,consensus,45.0,female,trunk,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
2433,HAM_0004653,ISIC_0028431,vasc,consensus,50.0,male,abdomen,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...


In [35]:
# train test split using dataframe

labels = table['dx'] #Extracts the column 'label' from the DataFrame

X_train, X_temp = train_test_split(table, test_size=0.2, stratify=labels, random_state = 42) #stratify=labels ช่วยให้  labels กระจายเท่าๆ กันทั้ง train&test
#X_train = 80% of df_dada || X_temp = 20% of df_dada

label_test_val = X_temp['dx']

X_test, X_val = train_test_split(X_temp, test_size=0.5, stratify=label_test_val, random_state = 42)
#X_test = 50% of X_temp || X_val = 50% of X_temp
print('The shape of train data',X_train.shape)
print('The shape of test data',X_test.shape)
print('The shape of validation data',X_val.shape)

The shape of train data (644, 8)
The shape of test data (80, 8)
The shape of validation data (81, 8)


In [37]:
X_train

,lesion_id,image_id,dx,dx_type,age,sex,localization,Path_Name
1141,HAM_0003290,ISIC_0031735,df,histo,80.0,male,upper extremity,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
12,HAM_0004234,ISIC_0025984,bkl,histo,85.0,female,chest,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
2414,HAM_0004097,ISIC_0025677,vasc,consensus,50.0,female,abdomen,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
36,HAM_0000351,ISIC_0029559,bkl,histo,85.0,male,back,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
3066,HAM_0005858,ISIC_0028879,nv,follow_up,55.0,male,back,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
...,...,...,...,...,...,...,...,...
9776,HAM_0005389,ISIC_0024372,akiec,histo,70.0,male,lower extremity,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
2429,HAM_0006193,ISIC_0029877,vasc,consensus,80.0,male,lower extremity,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
2471,HAM_0002134,ISIC_0024431,bcc,histo,45.0,male,face,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
2980,HAM_0001940,ISIC_0028502,nv,follow_up,45.0,female,abdomen,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...


In [45]:
# X_train.to_csv('/content/gdrive/MyDrive/Hackathon/Table_train.csv')

In [38]:
X_test

,lesion_id,image_id,dx,dx_type,age,sex,localization,Path_Name
2348,HAM_0003771,ISIC_0026163,vasc,histo,50.0,female,hand,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
1258,HAM_0004720,ISIC_0028642,mel,histo,40.0,male,lower extremity,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
2547,HAM_0002717,ISIC_0031284,bcc,histo,75.0,male,face,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
2495,HAM_0004871,ISIC_0030261,bcc,histo,70.0,male,back,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
1300,HAM_0006521,ISIC_0025155,mel,histo,80.0,male,lower extremity,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
...,...,...,...,...,...,...,...,...
2523,HAM_0005309,ISIC_0026282,bcc,histo,50.0,male,upper extremity,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
1199,HAM_0002848,ISIC_0027598,df,consensus,60.0,female,lower extremity,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
109,HAM_0001417,ISIC_0032382,bkl,histo,55.0,male,face,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
1131,HAM_0002706,ISIC_0025954,df,histo,40.0,male,lower extremity,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...


In [46]:
# X_test.to_csv('/content/gdrive/MyDrive/Hackathon/Table_test.csv')

In [43]:
test = pd.concat([X_test,df_another])

In [44]:
test

,lesion_id,image_id,dx,dx_type,age,sex,localization,Path_Name
2348,HAM_0003771,ISIC_0026163,vasc,histo,50.0,female,hand,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
1258,HAM_0004720,ISIC_0028642,mel,histo,40.0,male,lower extremity,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
2547,HAM_0002717,ISIC_0031284,bcc,histo,75.0,male,face,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
2495,HAM_0004871,ISIC_0030261,bcc,histo,70.0,male,back,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
1300,HAM_0006521,ISIC_0025155,mel,histo,80.0,male,lower extremity,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
...,...,...,...,...,...,...,...,...
10010,HAM_0002867,ISIC_0033084,akiec,histo,40.0,male,abdomen,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
10011,HAM_0002867,ISIC_0033550,akiec,histo,40.0,male,abdomen,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
10012,HAM_0002867,ISIC_0033536,akiec,histo,40.0,male,abdomen,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
10013,HAM_0000239,ISIC_0032854,akiec,histo,80.0,male,face,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...


In [47]:
# test.to_csv('/content/gdrive/MyDrive/Hackathon/Table_test_All.csv')

In [39]:
X_val

,lesion_id,image_id,dx,dx_type,age,sex,localization,Path_Name
58,HAM_0000700,ISIC_0028052,bkl,histo,60.0,male,face,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
3001,HAM_0007339,ISIC_0031536,nv,follow_up,50.0,male,upper extremity,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
1279,HAM_0004126,ISIC_0030211,mel,histo,80.0,female,back,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
1236,HAM_0001747,ISIC_0029575,mel,histo,45.0,male,back,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
9724,HAM_0007354,ISIC_0027719,akiec,histo,60.0,male,back,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
...,...,...,...,...,...,...,...,...
2420,HAM_0005938,ISIC_0025596,vasc,consensus,45.0,female,abdomen,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
2989,HAM_0006830,ISIC_0032285,nv,follow_up,45.0,female,abdomen,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
2360,HAM_0000940,ISIC_0032692,vasc,histo,35.0,female,lower extremity,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...
2534,HAM_0002346,ISIC_0027365,bcc,histo,70.0,male,face,/content/gdrive/MyDrive/Hackathon/Skin Cancer/...


In [48]:
# X_val.to_csv('/content/gdrive/MyDrive/Hackathon/Table_val.csv')

# Effnet

# Install

In [ ]:
!pip install efficientnet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 1.9 MB/s eta 0:00:00


# **Loading Library**

In [ ]:
!pip install efficientnet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 1.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Basic
import os
from os import makedirs
from os import listdir
from shutil import copyfile
from random import seed
from random import random
import numpy as np
import pandas as pd
from PIL import Image, ImageFile

# visuals
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.image import imread
from PIL import Image

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,ConfusionMatrixDisplay

# Tensorflow
import tensorflow as tf
from tensorflow.keras import callbacks
from keras.callbacks import Callback
from keras import layers
from keras import models
from tensorflow.keras import optimizers
from keras.optimizers import Adam
from efficientnet.keras import center_crop_and_resize, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.15.0


In [ ]:
!pwd

/content


In [ ]:
%cd '/content/train/Scabies Lyme Disease and other Infestations and Bites'

/content/train/Scabies Lyme Disease and other Infestations and Bites
